In [ ]:
from pyrosm import OSM

pbf_file = r"Areas-of-interest-POIs\Area-of-study.pbf"

categories = [
    "bakery",
    "supermarket",
    "kiosk",
    "butcher",
    "convenience",
    "beverages",
    "chemist",
    "laundry",
    "stationery",
    "greengrocer",
    "general"
    # "greengrocer",
    # "pharmacy"
]

# Load the clipped PBF
osm = OSM(pbf_file)

# Extract POIs for your categories (nodes, ways, multipolygons)
pois = osm.get_pois(custom_filter={"shop": categories})

# save to GeoPackage or CSV
pois.to_file(r"Areas-of-interest-POIs\Essential-activities-pois.gpkg", layer='pois', driver="GPKG")

print(f"Extracted {len(pois)} POIs from clipped region") 

Extracted 2148 POIs from clipped region


In [2]:
pois.head(10)

,version,timestamp,changeset,lat,id,lon,visible,tags,addr:city,addr:country,...,url,website,craft,lottery,organic,second_hand,shop,tobacco,geometry,osm_type
0,21,1744393166,0.0,52.274864,59620829,10.521164,False,"{""brand"":""Rewe"",""brand:wikidata"":""Q16968817"",""...",Braunschweig,DE,...,None,https://www.rewe.de/,None,None,yes,None,supermarket,None,POINT (10.52116 52.27486),node
1,18,1728062785,0.0,52.267452,59620848,10.522809,False,"{""brand"":""Penny"",""brand:wikidata"":""Q284688"",""b...",Braunschweig,DE,...,None,None,None,None,None,None,supermarket,None,POINT (10.52281 52.26745),node
2,5,1688150928,0.0,52.271214,59663397,10.508481,False,"{""check_date"":""2023-06-30"",""wheelchair"":""limit...",None,None,...,None,None,None,None,None,None,supermarket,None,POINT (10.50848 52.27121),node
3,4,1745502054,0.0,51.804989,60149547,10.351561,False,"{""wheelchair"":""yes""}",None,None,...,None,None,None,None,None,None,bakery,None,POINT (10.35156 51.80499),node
4,3,1518866943,0.0,51.805103,60149551,10.351620,False,"{""wheelchair"":""yes""}",None,None,...,None,None,None,None,None,None,butcher,None,POINT (10.35162 51.8051),node
5,12,1759333324,0.0,51.803360,60158405,10.336911,False,"{""addr:suburb"":""Clausthal-Zellerfeld"",""check_d...",Clausthal-Zellerfeld,None,...,None,None,None,None,None,None,butcher,None,POINT (10.33691 51.80336),node
6,13,1763303736,0.0,52.258835,80239016,10.506357,False,"{""brand"":""Rewe"",""brand:wikidata"":""Q16968817"",""...",None,None,...,None,None,None,None,None,None,supermarket,None,POINT (10.50636 52.25883),node
7,6,1690182946,0.0,52.253086,84794641,10.508952,False,"{""check_date:opening_hours"":""2023-07-24"",""whee...",None,None,...,None,None,None,None,None,None,kiosk,None,POINT (10.50895 52.25309),node
8,16,1728321943,0.0,52.285542,88183184,10.455995,False,"{""brand"":""EDEKA"",""brand:wikidata"":""Q57450576"",...",Braunschweig,None,...,None,None,None,None,None,None,beverages,None,POINT (10.45599 52.28554),node
9,17,1728321943,0.0,52.285664,88183214,10.455662,False,"{""brand"":""sander's backstube"",""brand:wikidata""...",Braunschweig,DE,...,None,https://www.sanders-backstube.de,None,None,None,None,bakery,None,POINT (10.45566 52.28566),node


In [3]:
pois.columns

Index(['version', 'timestamp', 'changeset', 'lat', 'id', 'lon', 'visible',
       'tags', 'addr:city', 'addr:country', 'addr:housenumber',
       'addr:housename', 'addr:postcode', 'addr:street', 'email', 'name',
       'opening_hours', 'operator', 'phone', 'ref', 'url', 'website', 'craft',
       'lottery', 'organic', 'second_hand', 'shop', 'tobacco', 'geometry',
       'osm_type'],
      dtype='object')

In [4]:
pois['tags'].head(10)

0    {"brand":"Rewe","brand:wikidata":"Q16968817","...
1    {"brand":"Penny","brand:wikidata":"Q284688","b...
2    {"check_date":"2023-06-30","wheelchair":"limit...
3                                 {"wheelchair":"yes"}
4                                 {"wheelchair":"yes"}
5    {"addr:suburb":"Clausthal-Zellerfeld","check_d...
6    {"brand":"Rewe","brand:wikidata":"Q16968817","...
7    {"check_date:opening_hours":"2023-07-24","whee...
8    {"brand":"EDEKA","brand:wikidata":"Q57450576",...
9    {"brand":"sander's backstube","brand:wikidata"...
Name: tags, dtype: object

In [5]:
import plotly.express as px

plot_gdf = pois.copy()

plot_gdf = plot_gdf.to_crs(epsg=25832)

plot_gdf['geometry'] = plot_gdf.geometry.centroid

plot_gdf = plot_gdf.to_crs(epsg=4326)

plot_gdf['lon'] = plot_gdf.geometry.x
plot_gdf['lat'] = plot_gdf.geometry.y

plot_gdf = plot_gdf[['name', 'shop', 'lat', 'lon']]

fig = px.scatter_map(
    plot_gdf,
    lat='lat',
    lon='lon',
    hover_name='name',
    hover_data=['shop'],
    color='shop',          # color points by shop type
    zoom=12,
    height=600,
)

fig.update_layout(
    map_style="carto-positron",  # free basemap
    margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()